<a href="https://colab.research.google.com/github/rahulgundala007/NLP_text_summarization/blob/main/BART_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setup of BART Model**

In [1]:
pip install transformers datasets torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [2]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.5 MB/s eta 0:00:00


**Importing the data from Hugging face**

In [3]:
from datasets import load_dataset

dataset = load_dataset('Rahulgundala007/ToSdataset2')  # Replace with the path or name of your dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

**BART Model**

In [4]:
from transformers import BartTokenizer

model_name = 'facebook/bart-large'  # or another BART model variant
tokenizer = BartTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = tokenizer(examples['ToS_Detail'], max_length=1024, truncation=True, padding="max_length")
    outputs = tokenizer(examples['ToS_Summary'], max_length=128, truncation=True, padding="max_length")

    return {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'labels': outputs.input_ids
    }

# Apply the function to the dataset
dataset = dataset.map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

**Training the model**

In [5]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

model = BartForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,  # Adjust based on your GPU capabilities
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],  # Assuming dataset split into train and validation
    eval_dataset=dataset['validation']
)

trainer.train()


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,4.065806
2,No log,3.186705
3,No log,2.872536


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=99, training_loss=4.817935519748264, metrics={'train_runtime': 202.6204, 'train_samples_per_second': 0.962, 'train_steps_per_second': 0.489, 'total_flos': 422585397411840.0, 'train_loss': 4.817935519748264, 'epoch': 3.0})

**Saving the trained model**

In [6]:
model.save_pretrained('./bart_summarizer')
tokenizer.save_pretrained('./bart_summarizer')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./bart_summarizer/tokenizer_config.json',
 './bart_summarizer/special_tokens_map.json',
 './bart_summarizer/vocab.json',
 './bart_summarizer/merges.txt',
 './bart_summarizer/added_tokens.json')

In [7]:
pip install PyPDF2 transformers sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.6 MB/s eta 0:00:00


In [28]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:  # Ensure there is text on the page
                text += page_text + ' '  # Append text of each page and a space
    return text

In [41]:
# Specify the path to your PDF file
pdf_path = '/content/samsung.pdf'

# Call the function with the PDF path
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)

Samsung Business Services Terms and Conditions  (USA)  
The sale and performance of Samsung Business Services  is governed by these 
Samsung Business Services Terms and Conditions (the "Terms and 
Conditions"). The Terms and Conditions apply to  all: 1) enterprise purchases 
of S ervices  (as defined below)  directly from Samsung; and 2) enterprise 
purchases  of Services t hrough a n authorized  Samsung reseller  or distributor  
(“reseller”) .  
 
By placing your O rder (as defined below)  for the Services, receiving delivery 
of the Services, utilizing the Services , or clicking/checking any  “I Agree” 
button or box or similar on the Samsung Business Services w ebsite in 
connection with your purchase , you agree to be bound by th ese Terms and 
Conditions .  
 
These Terms and Conditions are agreed  between (i) you, either  individually if 
you are agreeing to it in your own capacity , or if you are authorized to agree 
to these terms on behalf of your company or another entity, b

In [42]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_path = './bart_summarizer'  # Path to your trained BART model
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)

def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    output_tokens = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=5000,  # Adjust based on expected summary length
        min_length=500,
        num_beams=5,
        early_stopping=True
    )
    summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return summary


In [43]:
import sacrebleu

def compute_bleu(generated_summary, reference_summary):
    bleu_score = sacrebleu.corpus_bleu([generated_summary], [[reference_summary]])
    return bleu_score.score


In [57]:
pdf_path = '/content/samsung.pdf'
pdf_text = extract_text_from_pdf(pdf_path)
generated_summary = generate_summary(pdf_text)
print("Generated Summary:", generated_summary)

# You must provide this
reference_summary = "Certainly! Here's the text without quotes: The following are the terms and conditions that govern the sale and performance of Samsung Business Services, which include the provision of services directly from Samsung and through authorized resellers. The term Services refers to each component of the Samsung enterprise service offering, as described in a service guide (each, a service order), with specific terms for each component. These terms and conditions apply to all enterprise purchases of services from Samsung and reseller purchases of services through Samsung's authorized reseller, with final pricing and sales terms between the reseller and the purchaser determined by the seller. The Services and Deliverables are intended for internal business use unless permitted by Samsung in writing. Here are the key points to keep in mind: 1. Service Description: This section describes the services and deliverables offered by Samsung. 2. Payment Terms: These terms are determined by Samsung, with specific payment terms for services purchased directly from the company. 3. Availability: The terms used in this section are specific to the service guide or Order, with the meanings defined by the specific Order. 4. Service Availability: Each Samsung Service Guide or Order describes: (i) Available service, and (ii) Deliverables. 5. Terms Not Specified: Capitalized terms used but not defined in the terms and Conditions shall have the meanings as set forth in the applicable Order. 6. Service Terms: Here's a summary of the key terms used within the terms, including service descriptions, payment terms, and terms for service purchases, including reseller terms and terms applicable to Samsung's service offerings. 7. Business Services: The sale and performance of Samsung Services is governed by these terms, which are agreed between you and Samsung in your own capacity, between the entity for whose benefit you act, and Samsung Electronics America, Inc. (Samsung), which is responsible for service provision and service management. 8. Business Use: The Services are meant for internal use, with limited external use for external use. 9. Payment Terms: The payment terms that apply to you if Services are purchased through a reseller differ from those for directly purchased from Samsung. 10. Services Availability: Includes the terms for individual purchases, with detailed descriptions of the services provided by Samsung within the Service Guide and Order. 11. Services: Includes a detailed description of each component, including pricing, delivery, and use terms, applicable to all users, within the scope of the service agreement."
bleu_score = compute_bleu(generated_summary, reference_summary)
print(f"BLEU score: {bleu_score}")


Generated Summary: Samsung Business Services Terms and Conditions (the "Terms and Conditions") govern the sale and performance of Samsung Business Services, including the provision of services directly from Samsung and through authorized resellers. As used herein, “ Services” means each component of the Samsung enterprise service offering as described in a service guide (each, a service order), with specific terms for each component. The terms and conditions apply to all: 1) enterprise purchases of S ervices from Samsung; 2) reseller purchases of Services through Samsung's authorized reseller, with final pricing and sales terms between the reseller and the purchaser determined by the seller. The Services and Deliverables are intended for internal business use unless permitted by Samsung in writing. Here are the key points: 1. Service Description: This section describes the services and deliverables offered by Samsung. 2. Payment Terms: These terms are determined by Samsung, with specif